In [1]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
from sklearn import preprocessing

In [2]:
class QueenGenetic: 
    def __init__(self,init_pop_size, board_size):
        self.init_pop_size = init_pop_size
        self.pop_size = init_pop_size
        self.board_size = board_size
        self.total_cost = 0
        self.fitness_arr = np.zeros((self.pop_size,1))
        self.fitness_prob = np.zeros((self.pop_size,1))
        
    def PopulationInit(self):
        rand_pop = np.array(random.sample(range(self.board_size),self.board_size))
        self.pop = np.array(rand_pop)
        for index in range(self.init_pop_size-1):
            rand_pop = np.array(random.sample(range(self.board_size),self.board_size))
            self.pop = np.vstack((self.pop, rand_pop))
        #self.new_pop = np.array(self.pop) 
            
    #Fitness function: Calculates the overall cost
    def Fitness(self):
        self.fitness_arr = np.zeros((self.pop_size,1))
        #Check diagonal and reverse-diagonal conflicts
        for ii in range(self.pop_size): 
            curr_pop = self.pop[ii,:]
            curr_pop_arr = np.zeros((self.board_size,self.board_size))
            curr_pop_arr[np.arange(self.board_size), curr_pop] = 1
            for jj in range(self.board_size):
                diag_ones = np.count_nonzero(curr_pop_arr.diagonal(jj))
                rev_diag_ones = np.count_nonzero(np.fliplr(curr_pop_arr).diagonal(jj))
                if diag_ones > 1:
                    self.fitness_arr[ii] += (diag_ones) * (diag_ones - 1)/ 2
                if rev_diag_ones > 1:
                    self.fitness_arr[ii] += (rev_diag_ones) * (rev_diag_ones - 1)/ 2
            #Exponential is used to enhance parents with higher fitness values to be matched
            #Other approaches may be applied
            #If exponential returns 0, then optimal situation reached
            #Else then, more conflict will result in an exponential decrease of fitness value
            #+++Less fitness value for more conflicts => Less chance to be selected as parents
            self.fitness_arr[ii] = 1/(np.exp(self.fitness_arr[ii]))
        norm = np.sum(self.fitness_arr)
        self.fitness_prob = self.fitness_arr / norm
        self.fitness_prob = self.fitness_prob.reshape((self.fitness_prob.shape[0],))
        
        return self.fitness_arr
    
    def Permutation(self):
        rand_ind = random.sample(range(self.board_size), 2)
        min_rand_ind = np.min(rand_ind)
        max_rand_ind = np.max(rand_ind)
        diff = max_rand_ind - min_rand_ind
        while diff <= 1:
            rand_ind = random.sample(range(self.board_size), 2)
            min_rand_ind = np.min(rand_ind)
            max_rand_ind = np.max(rand_ind)
            diff = max_rand_ind - min_rand_ind
        return min_rand_ind, max_rand_ind, diff
    
    # Order 1 Crossover function
    # Every generation contains k new members
    # Parent choice counter check 
    def Crossover(self, k):  
        
        parents_ind = np.random.choice(np.arange(self.pop_size), 2, replace = False, p = self.fitness_prob)
        parents = np.array([self.pop[parents_ind[0]], self.pop[parents_ind[1]]])
        min_rand_ind, max_rand_ind, diff = self.Permutation()
        seq = parents[0,min_rand_ind:max_rand_ind] 
        parents_1 = parents[1][~np.isin(parents[1],seq)]
        new_pop = np.concatenate((parents_1[0:min_rand_ind],seq), axis=None)
        new_pop = np.concatenate((new_pop, parents_1[min_rand_ind:]), axis=None)
        
        for ii in range(k-1):
            parents_ind = np.random.choice(np.arange(self.pop_size), 2, replace = False, p = self.fitness_prob)
            parents = np.array([self.pop[parents_ind[0]], self.pop[parents_ind[1]]])
            min_rand_ind, max_rand_ind, diff = self.Permutation()
            seq = parents[0,min_rand_ind:max_rand_ind]            
            parents_1 = parents[1][~np.isin(parents[1],seq)]
            child = np.concatenate((parents_1[0:min_rand_ind],seq), axis=None)
            child = np.concatenate((child, parents_1[min_rand_ind:self.board_size]), axis=None)
            new_pop = np.vstack((new_pop, child))
            
        self.pop = np.copy(new_pop)
        self.pop_size = k
        
    # Mutation: Single and double mutation 
    def Mutation(self):
        for ii in range(self.pop_size):
            prob = np.random.uniform()
            #Double mutation
            if prob < 0.2:
                rand_ind = random.sample(range(self.board_size), 4)
                temp = self.pop[ii, rand_ind[1]]
                self.pop[ii, rand_ind[1]] = self.pop[ii, rand_ind[0]]
                self.pop[ii, rand_ind[0]] = temp
                
                temp = self.pop[ii, rand_ind[3]]
                self.pop[ii, rand_ind[3]] = self.pop[ii, rand_ind[2]]
                self.pop[ii, rand_ind[2]] = temp
            #Single Mutation    
            elif prob < 0.4:
                rand_ind = random.sample(range(self.board_size), 2)
                temp = self.pop[ii, rand_ind[1]]
                self.pop[ii, rand_ind[1]] = self.pop[ii, rand_ind[0]]
                self.pop[ii, rand_ind[0]] = temp
                
        
        
        
        

In [3]:
# N-Queens: Main constraint -> Columns are already set, only row-wise and diagonal conflicts

init_pop_size = int(input('Please enter size of initial population for N-Queens: '))
init_board_size = int(input('Please enter size of board for N-Queens: '))
board = QueenGenetic(init_pop_size, init_board_size)
board.PopulationInit()

gen = 0
fitness_arr = board.Fitness()
while True :
    print(gen, "th Generation with Average Fitness value =",np.mean(fitness_arr),"and Maximum Fitness Value =", np.max(fitness_arr))
    if np.max(fitness_arr) == 1:
        break
    board.Crossover(2048)
    board.Mutation()
    gen += 1
    fitness_arr = board.Fitness()



Please enter size of initial population for N-Queens: 1000
Please enter size of board for N-Queens: 100
0 th Generation with Average Fitness value = 1.3394475705023365e-10 and Maximum Fitness Value = 4.139937718785167e-08
1 th Generation with Average Fitness value = 3.6192448642794664e-08 and Maximum Fitness Value = 1.670170079024566e-05
2 th Generation with Average Fitness value = 1.6215194918917666e-06 and Maximum Fitness Value = 0.00033546262790251185
3 th Generation with Average Fitness value = 4.50611545757934e-05 and Maximum Fitness Value = 0.0024787521766663585
4 th Generation with Average Fitness value = 0.0002474813833014301 and Maximum Fitness Value = 0.006737946999085467
5 th Generation with Average Fitness value = 0.0007113891606797813 and Maximum Fitness Value = 0.049787068367863944
6 th Generation with Average Fitness value = 0.0017758199350950672 and Maximum Fitness Value = 0.049787068367863944
7 th Generation with Average Fitness value = 0.0034218923797267415 and Maximu